# Assessing fire risk by location in NYC

## Background
Using data provided by NYC OpenData, this notebook walks through the steps of analyzing fire risk in New York City.

## Import Libraries

In [1]:
# Data analysis and visualization
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

# Interactive maps
import folium
from folium.plugins import HeatMap

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline

## Load and describe data

Note: Data was filtered on the NYC OpenData site to only include incident classification groups that were fire-related (Structural and NonStructural Fires) prior to export.

In [2]:
# Connection to azure database 
# import pandas as pd, pyodbc
# server = 'finalprojectdata.database.windows.net'
# database = 'v2-project-data'
# username = 'finalproject1_pmprybylski'
# password = 'firedispatch1!'
# driver= '{ODBC Driver 17 for SQL Server}'
# # con_string = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password
# # con_string = 'DRIVER={SQL Server};SERVER='+ <server> +';DATABASE=' + <database>
# # cnxn = pyodbc.connect(con_string)
# cnxn = pyodbc.connect(
#     'DRIVER={ODBC Driver 17 for SQL Server};'
#     'SERVER=finalprojectdata.database.windows.net;'
#     'PORT=1433;'
#     'DATABASE=v2-project-data;'
#     'UID=finalproject1_pmprybylski;'
#     'PWD=firedispatch1!;'
# )
# query = """
# SELECT TOP 3 * FROM cleaned_fire_dispatch_data
# """
# result_port_map = pd.read_sql(query, cnxn)
# result_port_map

In [3]:
# Load the data into Python

# Fire Incident Dispatch
alarms_df = pd.read_csv('../data/raw/In-Service_Alarm_Box_Locations.csv')
dispatch_df = pd.read_csv('../data/raw/Fire_Incident_Dispatch_Data.csv')

# NYPD Complaints
nypd_df = pd.read_csv('../data/raw/NYPD_Complaint_18-21.csv')

# Dept of Buildings/Environ Control Board Violations
DOB18_df = pd.read_csv('../data/raw/DOB_ECB_Violations_18.csv')
DOB19_df = pd.read_csv('../data/raw/DOB_ECB_Violations_19.csv')
DOB20_df = pd.read_csv('../data/raw/DOB_ECB_Violations_20.csv')
DOB21_df = pd.read_csv('../data/raw/DOB_ECB_Violations_21.csv')

# Housing Maintenance Code Violations
codev_df = pd.read_csv('../data/raw/Housing_Maintenance_Code_Violations_18-21.csv')

# Orders to repair/vacate
vacate_df = pd.read_csv('../data/raw/Order_to_Repair_Vacate_18-21.csv')

## ELT

### Fire Incident Dispatches

In [4]:
# Join Fire Dispatch files
fires_df = pd.merge(left=alarms_df, right=dispatch_df, left_on='LOCATION', right_on='ALARM_BOX_LOCATION')
fires_df.head()

,BOROBOX,BOX_TYPE,LOCATION,ZIP,BOROUGH,COMMUNITYDISTICT,CITYCOUNCIL,LATITUDE,LONGITUDE,Location Point,...,FIRST_ACTIVATION_DATETIME,FIRST_ON_SCENE_DATETIME,INCIDENT_CLOSE_DATETIME,VALID_DISPATCH_RSPNS_TIME_INDC,VALID_INCIDENT_RSPNS_TIME_INDC,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY
0,B2653,ERS,3 AVE & 65 ST,11220.0,Brooklyn,BK07,38.0,40.63932,-74.023549,POINT (-74.02354939 40.63932033),...,02/06/2018 03:43:00 AM,02/06/2018 03:46:35 AM,02/06/2018 04:48:21 AM,N,Y,363,357,2,2,0
1,B2653,ERS,3 AVE & 65 ST,11220.0,Brooklyn,BK07,38.0,40.63932,-74.023549,POINT (-74.02354939 40.63932033),...,07/13/2018 01:56:41 PM,07/13/2018 01:59:36 PM,07/13/2018 02:09:18 PM,N,Y,227,194,3,2,1
2,B2653,ERS,3 AVE & 65 ST,11220.0,Brooklyn,BK07,38.0,40.63932,-74.023549,POINT (-74.02354939 40.63932033),...,08/06/2018 06:33:22 AM,08/06/2018 06:35:39 AM,08/06/2018 06:54:41 AM,N,Y,191,155,3,2,1
3,B2653,ERS,3 AVE & 65 ST,11220.0,Brooklyn,BK07,38.0,40.63932,-74.023549,POINT (-74.02354939 40.63932033),...,09/13/2018 06:28:56 PM,09/13/2018 06:30:12 PM,09/13/2018 07:41:08 PM,N,Y,160,95,4,2,1
4,B2653,ERS,3 AVE & 65 ST,11220.0,Brooklyn,BK07,38.0,40.63932,-74.023549,POINT (-74.02354939 40.63932033),...,09/16/2018 09:52:58 AM,09/16/2018 09:54:42 AM,09/16/2018 10:16:05 AM,N,Y,171,121,3,2,1


In [5]:
# Remove unnecessary columns
fires_df = fires_df[['STARFIRE_INCIDENT_ID',
               'INCIDENT_DATETIME',
               'ALARM_BOX_BOROUGH',
               'BOROBOX',
               'ALARM_BOX_LOCATION',
               'LATITUDE',
               'LONGITUDE',
               'INCIDENT_BOROUGH',
               'ZIPCODE',
               'INCIDENT_CLASSIFICATION',
               'INCIDENT_CLASSIFICATION_GROUP',
               'DISPATCH_RESPONSE_SECONDS_QY',
               'INCIDENT_RESPONSE_SECONDS_QY',
               'INCIDENT_TRAVEL_TM_SECONDS_QY',
               'ENGINES_ASSIGNED_QUANTITY',
               'LADDERS_ASSIGNED_QUANTITY',
               'OTHER_UNITS_ASSIGNED_QUANTITY',]]
fires_df.head()

,STARFIRE_INCIDENT_ID,INCIDENT_DATETIME,ALARM_BOX_BOROUGH,BOROBOX,ALARM_BOX_LOCATION,LATITUDE,LONGITUDE,INCIDENT_BOROUGH,ZIPCODE,INCIDENT_CLASSIFICATION,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY
0,1803726530140110,02/06/2018 03:40:32 AM,BROOKLYN,B2653,3 AVE & 65 ST,40.63932,-74.023549,BROOKLYN,11220.0,Automobile Fire,NonStructural Fires,6,363,357,2,2,0
1,1819426530140570,07/13/2018 01:55:49 PM,BROOKLYN,B2653,3 AVE & 65 ST,40.63932,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Other fire,Structural Fires,33,227,194,3,2,1
2,1821826530140150,08/06/2018 06:32:28 AM,BROOKLYN,B2653,3 AVE & 65 ST,40.63932,-74.023549,BROOKLYN,11220.0,Demolition Debris or Rubbish Fire,NonStructural Fires,36,191,155,3,2,1
3,1825626530241090,09/13/2018 06:27:32 PM,BROOKLYN,B2653,3 AVE & 65 ST,40.63932,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Other fire,Structural Fires,65,160,95,4,2,1
4,1825926530240410,09/16/2018 09:51:51 AM,BROOKLYN,B2653,3 AVE & 65 ST,40.63932,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Food on the stove fire,Structural Fires,50,171,121,3,2,1


In [6]:
# Export cleaned data to csv for visualization use
fires_df.to_csv('../data/processed/cleaned_fire_dispatch_data.csv', index=False)

In [7]:
fires_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62111 entries, 0 to 62110
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   STARFIRE_INCIDENT_ID           62111 non-null  int64  
 1   INCIDENT_DATETIME              62111 non-null  object 
 2   ALARM_BOX_BOROUGH              62111 non-null  object 
 3   BOROBOX                        62111 non-null  object 
 4   ALARM_BOX_LOCATION             62111 non-null  object 
 5   LATITUDE                       62111 non-null  float64
 6   LONGITUDE                      62111 non-null  float64
 7   INCIDENT_BOROUGH               62111 non-null  object 
 8   ZIPCODE                        61221 non-null  float64
 9   INCIDENT_CLASSIFICATION        62111 non-null  object 
 10  INCIDENT_CLASSIFICATION_GROUP  62111 non-null  object 
 11  DISPATCH_RESPONSE_SECONDS_QY   62111 non-null  int64  
 12  INCIDENT_RESPONSE_SECONDS_QY   62111 non-null 

In [8]:
# Convert INCIDENT_DATETIME column to datetime
fires_df['INCIDENT_DATETIME'] = fires_df['INCIDENT_DATETIME'].apply(lambda x: dt.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
fires_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62111 entries, 0 to 62110
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   STARFIRE_INCIDENT_ID           62111 non-null  int64         
 1   INCIDENT_DATETIME              62111 non-null  datetime64[ns]
 2   ALARM_BOX_BOROUGH              62111 non-null  object        
 3   BOROBOX                        62111 non-null  object        
 4   ALARM_BOX_LOCATION             62111 non-null  object        
 5   LATITUDE                       62111 non-null  float64       
 6   LONGITUDE                      62111 non-null  float64       
 7   INCIDENT_BOROUGH               62111 non-null  object        
 8   ZIPCODE                        61221 non-null  float64       
 9   INCIDENT_CLASSIFICATION        62111 non-null  object        
 10  INCIDENT_CLASSIFICATION_GROUP  62111 non-null  object        
 11  DISPATCH_RESPON

In [9]:
# Add a column that splits off the year
fires_df['YEAR'] = fires_df['INCIDENT_DATETIME'].dt.year

# Move that column to the beginning of the frame
year = fires_df['YEAR']
fires_df.drop(labels=['YEAR'], axis=1, inplace=True)
fires_df.insert(0,'YEAR', year)
fires_df.head()

,YEAR,STARFIRE_INCIDENT_ID,INCIDENT_DATETIME,ALARM_BOX_BOROUGH,BOROBOX,ALARM_BOX_LOCATION,LATITUDE,LONGITUDE,INCIDENT_BOROUGH,ZIPCODE,INCIDENT_CLASSIFICATION,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY
0,2018,1803726530140110,2018-02-06 03:40:32,BROOKLYN,B2653,3 AVE & 65 ST,40.63932,-74.023549,BROOKLYN,11220.0,Automobile Fire,NonStructural Fires,6,363,357,2,2,0
1,2018,1819426530140570,2018-07-13 13:55:49,BROOKLYN,B2653,3 AVE & 65 ST,40.63932,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Other fire,Structural Fires,33,227,194,3,2,1
2,2018,1821826530140150,2018-08-06 06:32:28,BROOKLYN,B2653,3 AVE & 65 ST,40.63932,-74.023549,BROOKLYN,11220.0,Demolition Debris or Rubbish Fire,NonStructural Fires,36,191,155,3,2,1
3,2018,1825626530241090,2018-09-13 18:27:32,BROOKLYN,B2653,3 AVE & 65 ST,40.63932,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Other fire,Structural Fires,65,160,95,4,2,1
4,2018,1825926530240410,2018-09-16 09:51:51,BROOKLYN,B2653,3 AVE & 65 ST,40.63932,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Food on the stove fire,Structural Fires,50,171,121,3,2,1


In [10]:
# Split into training(18-19) and cross-validation(20-21) dataframes
fires1 = fires_df.loc[(fires_df.YEAR == 2018)|(fires_df.YEAR == 2019)]
fires1

,YEAR,STARFIRE_INCIDENT_ID,INCIDENT_DATETIME,ALARM_BOX_BOROUGH,BOROBOX,ALARM_BOX_LOCATION,LATITUDE,LONGITUDE,INCIDENT_BOROUGH,ZIPCODE,INCIDENT_CLASSIFICATION,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY
0,2018,1803726530140110,2018-02-06 03:40:32,BROOKLYN,B2653,3 AVE & 65 ST,40.639320,-74.023549,BROOKLYN,11220.0,Automobile Fire,NonStructural Fires,6,363,357,2,2,0
1,2018,1819426530140570,2018-07-13 13:55:49,BROOKLYN,B2653,3 AVE & 65 ST,40.639320,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Other fire,Structural Fires,33,227,194,3,2,1
2,2018,1821826530140150,2018-08-06 06:32:28,BROOKLYN,B2653,3 AVE & 65 ST,40.639320,-74.023549,BROOKLYN,11220.0,Demolition Debris or Rubbish Fire,NonStructural Fires,36,191,155,3,2,1
3,2018,1825626530241090,2018-09-13 18:27:32,BROOKLYN,B2653,3 AVE & 65 ST,40.639320,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Other fire,Structural Fires,65,160,95,4,2,1
4,2018,1825926530240410,2018-09-16 09:51:51,BROOKLYN,B2653,3 AVE & 65 ST,40.639320,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Food on the stove fire,Structural Fires,50,171,121,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62098,2019,1933638740140327,2019-12-02 09:19:19,BROOKLYN,B3874,AVENUE T & W 13 ST,40.598038,-73.984494,BROOKLYN,11223.0,Manhole Fire - Seeping Smoke,NonStructural Fires,40,415,375,3,2,1
62099,2019,1934638740141089,2019-12-12 21:37:32,BROOKLYN,B3874,AVENUE T & W 13 ST,40.598038,-73.984494,BROOKLYN,11223.0,Automobile Fire,NonStructural Fires,21,382,361,1,1,0
62104,2018,1824547130150240,2018-09-02 09:37:33,QUEENS,Q4713,MERRICK BLVD & BAISLEY BLVD,40.683778,-73.769742,QUEENS,11434.0,Store Fire,Structural Fires,30,252,222,4,2,1
62105,2019,1916447130150628,2019-06-13 19:02:10,QUEENS,Q4713,MERRICK BLVD & BAISLEY BLVD,40.683778,-73.769742,QUEENS,11434.0,Demolition Debris or Rubbish Fire,NonStructural Fires,32,262,230,1,1,0


In [11]:
fires2 = fires_df.loc[(fires_df.YEAR == 2020)|(fires_df.YEAR == 2021)]
fires2

,YEAR,STARFIRE_INCIDENT_ID,INCIDENT_DATETIME,ALARM_BOX_BOROUGH,BOROBOX,ALARM_BOX_LOCATION,LATITUDE,LONGITUDE,INCIDENT_BOROUGH,ZIPCODE,INCIDENT_CLASSIFICATION,INCIDENT_CLASSIFICATION_GROUP,DISPATCH_RESPONSE_SECONDS_QY,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,ENGINES_ASSIGNED_QUANTITY,LADDERS_ASSIGNED_QUANTITY,OTHER_UNITS_ASSIGNED_QUANTITY
10,2020,2000626530140006,2020-01-06 00:13:36,BROOKLYN,B2653,3 AVE & 65 ST,40.639320,-74.023549,BROOKLYN,11220.0,Automobile Fire,NonStructural Fires,39,170,131,1,1,0
11,2020,2001626530140827,2020-01-16 17:18:31,BROOKLYN,B2653,3 AVE & 65 ST,40.639320,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Food on the stove fire,Structural Fires,28,179,151,3,2,1
12,2020,2016226530140729,2020-06-10 17:31:54,BROOKLYN,B2653,3 AVE & 65 ST,40.639320,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Food on the stove fire,Structural Fires,42,197,155,3,2,1
13,2020,2025726530140895,2020-09-13 18:22:48,BROOKLYN,B2653,3 AVE & 65 ST,40.639320,-74.023549,BROOKLYN,11220.0,Demolition Debris or Rubbish Fire,NonStructural Fires,42,174,132,3,2,1
14,2020,2033426530140121,2020-11-29 03:54:56,BROOKLYN,B2653,3 AVE & 65 ST,40.639320,-74.023549,BROOKLYN,11220.0,Multiple Dwelling 'A' - Other fire,Structural Fires,15,140,125,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62103,2021,2105240000000000,2021-02-21 15:15:00,BROOKLYN,B3874,AVENUE T & W 13 ST,40.598038,-73.984494,BROOKLYN,11223.0,Manhole Fire - Other,NonStructural Fires,32,382,350,3,2,1
62107,2020,2000347130150135,2020-01-03 04:53:38,QUEENS,Q4713,MERRICK BLVD & BAISLEY BLVD,40.683778,-73.769742,QUEENS,11434.0,Private Dwelling Fire,Structural Fires,22,236,214,3,3,1
62108,2020,2008047130150389,2020-03-20 17:08:53,QUEENS,Q4713,MERRICK BLVD & BAISLEY BLVD,40.683778,-73.769742,QUEENS,11434.0,Automobile Fire,NonStructural Fires,24,229,205,1,1,0
62109,2020,2020147130150874,2020-07-19 23:05:15,QUEENS,Q4713,MERRICK BLVD & BAISLEY BLVD,40.683778,-73.769742,QUEENS,11434.0,Private Dwelling Fire,Structural Fires,34,234,200,3,2,1


### NYPD Complaints

In [12]:
nypd_df

,CMPLNT_FR_DT,BORO_NM,CMPLNT_NUM
0,01/01/2018 12:00:00 AM,BRONX,8337
1,01/01/2018 12:00:00 AM,BROOKLYN,11193
2,01/01/2018 12:00:00 AM,MANHATTAN,9614
3,01/01/2018 12:00:00 AM,QUEENS,7450
4,01/01/2018 12:00:00 AM,STATEN ISLAND,1677
...,...,...,...
211,12/01/2020 12:00:00 AM,BROOKLYN,8631
212,12/01/2020 12:00:00 AM,MANHATTAN,7200
213,12/01/2020 12:00:00 AM,QUEENS,6697
214,12/01/2020 12:00:00 AM,STATEN ISLAND,1216


In [13]:
# Rename columns for easier useage
nypd_df.rename(columns={
    'CMPLNT_FR_DT':'COMPLAINT_DATE',
    'BORO_NM':'BOROUGH',
    'CMPLNT_NUM':'NUMBER_OF_COMPLAINTS'
}, inplace=True)
nypd_df.head()

,COMPLAINT_DATE,BOROUGH,NUMBER_OF_COMPLAINTS
0,01/01/2018 12:00:00 AM,BRONX,8337
1,01/01/2018 12:00:00 AM,BROOKLYN,11193
2,01/01/2018 12:00:00 AM,MANHATTAN,9614
3,01/01/2018 12:00:00 AM,QUEENS,7450
4,01/01/2018 12:00:00 AM,STATEN ISLAND,1677


In [14]:
# Export cleaned data to csv for visualization use
nypd_df.to_csv('../data/processed/cleaned_nypd_complaint_data.csv', index=False)

In [15]:
nypd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   COMPLAINT_DATE        216 non-null    object
 1   BOROUGH               180 non-null    object
 2   NUMBER_OF_COMPLAINTS  216 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 5.2+ KB


In [16]:
# Convert COMPLAINT_DATE column to datetime
nypd_df['COMPLAINT_DATE'] = nypd_df['COMPLAINT_DATE'].apply(lambda x: dt.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
nypd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   COMPLAINT_DATE        216 non-null    datetime64[ns]
 1   BOROUGH               180 non-null    object        
 2   NUMBER_OF_COMPLAINTS  216 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 5.2+ KB


In [17]:
# Add a column that splits off the year
nypd_df['YEAR'] = nypd_df['COMPLAINT_DATE'].dt.year

# Move that column to the beginning of the frame
year = nypd_df['YEAR']
nypd_df.drop(labels=['YEAR'], axis=1, inplace=True)
nypd_df.insert(0,'YEAR', year)
nypd_df.head()

,YEAR,COMPLAINT_DATE,BOROUGH,NUMBER_OF_COMPLAINTS
0,2018,2018-01-01,BRONX,8337
1,2018,2018-01-01,BROOKLYN,11193
2,2018,2018-01-01,MANHATTAN,9614
3,2018,2018-01-01,QUEENS,7450
4,2018,2018-01-01,STATEN ISLAND,1677


In [18]:
# Split into training(18-19) and cross-validation(20-21) dataframes
nypd1 = nypd_df.loc[(nypd_df.YEAR == 2018)|(nypd_df.YEAR == 2019)]
nypd1

,YEAR,COMPLAINT_DATE,BOROUGH,NUMBER_OF_COMPLAINTS
0,2018,2018-01-01,BRONX,8337
1,2018,2018-01-01,BROOKLYN,11193
2,2018,2018-01-01,MANHATTAN,9614
3,2018,2018-01-01,QUEENS,7450
4,2018,2018-01-01,STATEN ISLAND,1677
...,...,...,...,...
139,2019,2019-12-01,BROOKLYN,10331
140,2019,2019-12-01,MANHATTAN,9292
141,2019,2019-12-01,QUEENS,7481
142,2019,2019-12-01,STATEN ISLAND,1477


In [19]:
nypd2 = nypd_df.loc[(nypd_df.YEAR == 2020) | (nypd_df.YEAR == 2021)]
nypd2

,YEAR,COMPLAINT_DATE,BOROUGH,NUMBER_OF_COMPLAINTS
144,2020,2020-01-01,BRONX,8352
145,2020,2020-01-01,BROOKLYN,10810
146,2020,2020-01-01,MANHATTAN,9812
147,2020,2020-01-01,QUEENS,7821
148,2020,2020-01-01,STATEN ISLAND,1608
...,...,...,...,...
211,2020,2020-12-01,BROOKLYN,8631
212,2020,2020-12-01,MANHATTAN,7200
213,2020,2020-12-01,QUEENS,6697
214,2020,2020-12-01,STATEN ISLAND,1216


### Dept of Buildings/Environmental Control Board Violations

In [20]:
DOB18_df.head()

,BORO,VIOLATION_TYPE,DOB_VIOLATION_NUMBER
0,1,Boilers,342
1,1,Construction,14250
2,1,Cranes and Derricks,164
3,1,Elevators,2271
4,1,Local Law,1619


In [21]:
DOB19_df.head()

,BORO,VIOLATION_TYPE,DOB_VIOLATION_NUMBER
0,1,Boilers,78
1,1,Construction,14808
2,1,Cranes and Derricks,216
3,1,Elevators,1636
4,1,Local Law,68


In [22]:
DOB20_df.head()

,BORO,VIOLATION_TYPE,DOB_VIOLATION_NUMBER
0,1,Boilers,16
1,1,Construction,10053
2,1,Cranes and Derricks,127
3,1,Elevators,271
4,1,Local Law,570


In [23]:
DOB21_df.head()

,BORO,VIOLATION_TYPE,DOB_VIOLATION_NUMBER
0,4,Quality of Life,21
1,2,Site Safety,22
2,3,Zoning,32
3,5,Unknown,91
4,3,Boilers,5


In [24]:
# Add columns to designate years for each df
DOB18_df.insert(0, 'YEAR', '2018')
DOB19_df.insert(0, 'YEAR', '2019')
DOB20_df.insert(0, 'YEAR', '2020')
DOB21_df.insert(0, 'YEAR', '2021')

In [25]:
DOB21_df.head()

,YEAR,BORO,VIOLATION_TYPE,DOB_VIOLATION_NUMBER
0,2021,4,Quality of Life,21
1,2021,2,Site Safety,22
2,2021,3,Zoning,32
3,2021,5,Unknown,91
4,2021,3,Boilers,5


In [26]:
# Replace Borough Number with names based on file schema 
# 1 = Manhattan
# 2 = Bronx
# 3 = Brooklyn
# 4 = Queens
# 5 = Staten Island

def f(x):
    if x['BORO'] == 1: return 'MANHATTAN'
    elif x['BORO'] == 2: return 'BRONX'
    elif x['BORO'] == 3: return 'BROOKLYN'
    elif x['BORO'] == 4: return 'QUEENS'
    elif x['BORO'] == 5: return 'STATEN ISLAND'
    else: return ''

DOB18_df['BOROUGH'] = DOB18_df.apply(f, axis=1)
DOB19_df['BOROUGH'] = DOB19_df.apply(f, axis=1) 
DOB20_df['BOROUGH'] = DOB20_df.apply(f, axis=1)
DOB21_df['BOROUGH'] = DOB21_df.apply(f, axis=1)


In [27]:
# DOB18_df.head()
# DOB19_df.head()
# DOB20_df.head()
DOB21_df.head()

,YEAR,BORO,VIOLATION_TYPE,DOB_VIOLATION_NUMBER,BOROUGH
0,2021,4,Quality of Life,21,QUEENS
1,2021,2,Site Safety,22,BRONX
2,2021,3,Zoning,32,BROOKLYN
3,2021,5,Unknown,91,STATEN ISLAND
4,2021,3,Boilers,5,BROOKLYN


In [28]:
# Combine the 2018 and 2019 dataframes for training
dobv1= DOB18_df.append(DOB19_df)
dobv1

,YEAR,BORO,VIOLATION_TYPE,DOB_VIOLATION_NUMBER,BOROUGH
0,2018,1,Boilers,342,MANHATTAN
1,2018,1,Construction,14250,MANHATTAN
2,2018,1,Cranes and Derricks,164,MANHATTAN
3,2018,1,Elevators,2271,MANHATTAN
4,2018,1,Local Law,1619,MANHATTAN
...,...,...,...,...,...
51,2019,5,Quality of Life,6,STATEN ISLAND
52,2019,5,Signs,26,STATEN ISLAND
53,2019,5,Site Safety,11,STATEN ISLAND
54,2019,5,Unknown,134,STATEN ISLAND


In [29]:
# Combine the 2020 and 2021 dataframes for cross-validation of predictive algorithm
dobv2 = DOB20_df.append(DOB21_df)
dobv2

,YEAR,BORO,VIOLATION_TYPE,DOB_VIOLATION_NUMBER,BOROUGH
0,2020,1,Boilers,16,MANHATTAN
1,2020,1,Construction,10053,MANHATTAN
2,2020,1,Cranes and Derricks,127,MANHATTAN
3,2020,1,Elevators,271,MANHATTAN
4,2020,1,Local Law,570,MANHATTAN
...,...,...,...,...,...
43,2021,1,Boilers,2,MANHATTAN
44,2021,4,Boilers,0,QUEENS
45,2021,5,Site Safety,9,STATEN ISLAND
46,2021,4,Zoning,28,QUEENS


In [30]:
# Rename columns for easier usage
dobv1.rename(columns={
    'YEAR': 'YEAR_OF_COMPLAINT',
    'DOB_VIOLATION_NUMBER': 'NUMBER_OF_VIOLATIONS'
    }, inplace=True)
dobv1.head()

,YEAR_OF_COMPLAINT,BORO,VIOLATION_TYPE,NUMBER_OF_VIOLATIONS,BOROUGH
0,2018,1,Boilers,342,MANHATTAN
1,2018,1,Construction,14250,MANHATTAN
2,2018,1,Cranes and Derricks,164,MANHATTAN
3,2018,1,Elevators,2271,MANHATTAN
4,2018,1,Local Law,1619,MANHATTAN


In [31]:
dobv2.rename(columns={
    'YEAR': 'YEAR_OF_COMPLAINT',
    'DOB_VIOLATION_NUMBER': 'NUMBER_OF_VIOLATIONS'
    }, inplace=True)
dobv2.head()

,YEAR_OF_COMPLAINT,BORO,VIOLATION_TYPE,NUMBER_OF_VIOLATIONS,BOROUGH
0,2020,1,Boilers,16,MANHATTAN
1,2020,1,Construction,10053,MANHATTAN
2,2020,1,Cranes and Derricks,127,MANHATTAN
3,2020,1,Elevators,271,MANHATTAN
4,2020,1,Local Law,570,MANHATTAN


In [32]:
# Drop unnecessary columns
dobv1 = dobv1[['YEAR_OF_COMPLAINT',
                   'BOROUGH',
                   'VIOLATION_TYPE',
                   'NUMBER_OF_VIOLATIONS'
]]
dobv1.head()

,YEAR_OF_COMPLAINT,BOROUGH,VIOLATION_TYPE,NUMBER_OF_VIOLATIONS
0,2018,MANHATTAN,Boilers,342
1,2018,MANHATTAN,Construction,14250
2,2018,MANHATTAN,Cranes and Derricks,164
3,2018,MANHATTAN,Elevators,2271
4,2018,MANHATTAN,Local Law,1619


In [33]:
dobv2 = dobv2[['YEAR_OF_COMPLAINT',
                   'BOROUGH',
                   'VIOLATION_TYPE',
                   'NUMBER_OF_VIOLATIONS'
]]
dobv2.head()

,YEAR_OF_COMPLAINT,BOROUGH,VIOLATION_TYPE,NUMBER_OF_VIOLATIONS
0,2020,MANHATTAN,Boilers,16
1,2020,MANHATTAN,Construction,10053
2,2020,MANHATTAN,Cranes and Derricks,127
3,2020,MANHATTAN,Elevators,271
4,2020,MANHATTAN,Local Law,570


In [34]:
# Merge data for clean data file
dobv_df = dobv1.append(dobv2)

In [35]:
# Export cleaned data to csv for visualization use
dobv_df.to_csv('../data/processed/cleaned_dob_violations_data.csv', index=False)

### Housing Maintenance Code Violations

In [36]:
codev_df

,NOVIssuedDate,ViolationID,Borough
0,01/01/2018 12:00:00 AM,13430,BRONX
1,01/01/2018 12:00:00 AM,17188,BROOKLYN
2,01/01/2018 12:00:00 AM,8279,MANHATTAN
3,01/01/2018 12:00:00 AM,4191,QUEENS
4,01/01/2018 12:00:00 AM,545,STATEN ISLAND
...,...,...,...
200,05/01/2021 12:00:00 AM,8576,BRONX
201,05/01/2021 12:00:00 AM,13015,BROOKLYN
202,05/01/2021 12:00:00 AM,6341,MANHATTAN
203,05/01/2021 12:00:00 AM,2930,QUEENS


In [37]:
# Rename columns
codev_df.rename(columns={
    'NOVIssuedDate': 'VIOLATION_ISSUE_DATE',
    'ViolationID': 'NUMBER_OF_VIOLATIONS',
    'Borough': 'BOROUGH'
}, inplace=True)
codev_df.head()

,VIOLATION_ISSUE_DATE,NUMBER_OF_VIOLATIONS,BOROUGH
0,01/01/2018 12:00:00 AM,13430,BRONX
1,01/01/2018 12:00:00 AM,17188,BROOKLYN
2,01/01/2018 12:00:00 AM,8279,MANHATTAN
3,01/01/2018 12:00:00 AM,4191,QUEENS
4,01/01/2018 12:00:00 AM,545,STATEN ISLAND


In [38]:
# Export cleaned data to csv for visualization use
codev_df.to_csv('../data/processed/cleaned_housing_code_violation_data.csv', index=False)

In [39]:
codev_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   VIOLATION_ISSUE_DATE  205 non-null    object
 1   NUMBER_OF_VIOLATIONS  205 non-null    int64 
 2   BOROUGH               205 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.9+ KB


In [40]:
# Convert VIOLATION_ISSUE_DATE column to datetime
codev_df['VIOLATION_ISSUE_DATE'] = codev_df['VIOLATION_ISSUE_DATE'].apply(lambda x: dt.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
codev_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   VIOLATION_ISSUE_DATE  205 non-null    datetime64[ns]
 1   NUMBER_OF_VIOLATIONS  205 non-null    int64         
 2   BOROUGH               205 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 4.9+ KB


In [41]:
# Add a column that splits off the year
codev_df['YEAR'] = codev_df['VIOLATION_ISSUE_DATE'].dt.year

# Move that column to the beginning of the frame
year = codev_df['YEAR']
codev_df.drop(labels=['YEAR'], axis=1, inplace=True)
codev_df.insert(0,'YEAR', year)
codev_df.head()

,YEAR,VIOLATION_ISSUE_DATE,NUMBER_OF_VIOLATIONS,BOROUGH
0,2018,2018-01-01,13430,BRONX
1,2018,2018-01-01,17188,BROOKLYN
2,2018,2018-01-01,8279,MANHATTAN
3,2018,2018-01-01,4191,QUEENS
4,2018,2018-01-01,545,STATEN ISLAND


In [42]:
# Split into training(18-19) and cross-validation(20-21) dataframes
codev1 = codev_df.loc[(codev_df.YEAR == 2018)|(codev_df.YEAR == 2019)]
codev1

,YEAR,VIOLATION_ISSUE_DATE,NUMBER_OF_VIOLATIONS,BOROUGH
0,2018,2018-01-01,13430,BRONX
1,2018,2018-01-01,17188,BROOKLYN
2,2018,2018-01-01,8279,MANHATTAN
3,2018,2018-01-01,4191,QUEENS
4,2018,2018-01-01,545,STATEN ISLAND
...,...,...,...,...
115,2019,2019-12-01,14352,BRONX
116,2019,2019-12-01,18868,BROOKLYN
117,2019,2019-12-01,10057,MANHATTAN
118,2019,2019-12-01,4658,QUEENS


In [43]:
codev2 = codev_df.loc[(codev_df.YEAR == 2020) | (codev_df.YEAR == 2021)]
codev2

,YEAR,VIOLATION_ISSUE_DATE,NUMBER_OF_VIOLATIONS,BOROUGH
120,2020,2020-01-01,13876,BRONX
121,2020,2020-01-01,20237,BROOKLYN
122,2020,2020-01-01,10527,MANHATTAN
123,2020,2020-01-01,4635,QUEENS
124,2020,2020-01-01,521,STATEN ISLAND
...,...,...,...,...
200,2021,2021-05-01,8576,BRONX
201,2021,2021-05-01,13015,BROOKLYN
202,2021,2021-05-01,6341,MANHATTAN
203,2021,2021-05-01,2930,QUEENS


### Orders to Vacate

In [44]:
vacate_df

,VACATE EFFECTIVE DATE,BOROUGH,VACATE ORDER NUMBER
0,01/01/2018 12:00:00 AM,BK,30
1,01/01/2018 12:00:00 AM,BX,25
2,01/01/2018 12:00:00 AM,MN,13
3,01/01/2018 12:00:00 AM,QN,14
4,01/01/2018 12:00:00 AM,SI,7
...,...,...,...
190,05/01/2021 12:00:00 AM,BK,8
191,05/01/2021 12:00:00 AM,BX,6
192,05/01/2021 12:00:00 AM,MN,2
193,05/01/2021 12:00:00 AM,QN,6


In [45]:
# Rename columns
vacate_df.rename(columns={
    'VACATE EFFECTIVE DATE': 'VACATE_DATE',
    'BOROUGH': 'BORO',
    'VACATE ORDER NUMBER': 'NUMBER_OF_VACATE_ORDERS'
}, inplace=True)
vacate_df.head()

,VACATE_DATE,BORO,NUMBER_OF_VACATE_ORDERS
0,01/01/2018 12:00:00 AM,BK,30
1,01/01/2018 12:00:00 AM,BX,25
2,01/01/2018 12:00:00 AM,MN,13
3,01/01/2018 12:00:00 AM,QN,14
4,01/01/2018 12:00:00 AM,SI,7


In [46]:
# Replace Borough abbreviations with names based on file schema 
# MN = Manhattan
# BX = Bronx
# BK = Brooklyn
# QN = Queens
# SI = Staten Island

def f(x):
    if x['BORO'] == 'MN': return 'MANHATTAN'
    elif x['BORO'] == 'BX': return 'BRONX'
    elif x['BORO'] == 'BK': return 'BROOKLYN'
    elif x['BORO'] == 'QN': return 'QUEENS'
    elif x['BORO'] == 'SI': return 'STATEN ISLAND'
    else: return ''

vacate_df['BOROUGH'] = vacate_df.apply(f, axis=1)
vacate_df.head()

,VACATE_DATE,BORO,NUMBER_OF_VACATE_ORDERS,BOROUGH
0,01/01/2018 12:00:00 AM,BK,30,BROOKLYN
1,01/01/2018 12:00:00 AM,BX,25,BRONX
2,01/01/2018 12:00:00 AM,MN,13,MANHATTAN
3,01/01/2018 12:00:00 AM,QN,14,QUEENS
4,01/01/2018 12:00:00 AM,SI,7,STATEN ISLAND


In [47]:
# Drop unneeded columns
vacate_df = vacate_df[['VACATE_DATE',
                       'BOROUGH',
                       'NUMBER_OF_VACATE_ORDERS']]
vacate_df.head()

,VACATE_DATE,BOROUGH,NUMBER_OF_VACATE_ORDERS
0,01/01/2018 12:00:00 AM,BROOKLYN,30
1,01/01/2018 12:00:00 AM,BRONX,25
2,01/01/2018 12:00:00 AM,MANHATTAN,13
3,01/01/2018 12:00:00 AM,QUEENS,14
4,01/01/2018 12:00:00 AM,STATEN ISLAND,7


In [48]:
# Export cleaned data to csv for visualization use
vacate_df.to_csv('../data/processed/cleaned_order_to_vacate_data.csv', index=False)

In [49]:
vacate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   VACATE_DATE              195 non-null    object
 1   BOROUGH                  195 non-null    object
 2   NUMBER_OF_VACATE_ORDERS  195 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.7+ KB


In [50]:
# Convert VACATE_DATE column to datetime
vacate_df['VACATE_DATE'] = vacate_df['VACATE_DATE'].apply(lambda x: dt.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
vacate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   VACATE_DATE              195 non-null    datetime64[ns]
 1   BOROUGH                  195 non-null    object        
 2   NUMBER_OF_VACATE_ORDERS  195 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 4.7+ KB


/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [51]:
# Add a column that splits off the year
vacate_df['YEAR'] = vacate_df['VACATE_DATE'].dt.year

# Move that column to the beginning of the frame
year = vacate_df['YEAR']
vacate_df.drop(labels=['YEAR'], axis=1, inplace=True)
vacate_df.insert(0,'YEAR', year)
vacate_df.head()

,YEAR,VACATE_DATE,BOROUGH,NUMBER_OF_VACATE_ORDERS
0,2018,2018-01-01,BROOKLYN,30
1,2018,2018-01-01,BRONX,25
2,2018,2018-01-01,MANHATTAN,13
3,2018,2018-01-01,QUEENS,14
4,2018,2018-01-01,STATEN ISLAND,7


In [52]:
# Split into training(18-19) and cross-validation(20-21) dataframes
vacate1 = vacate_df.loc[(vacate_df.YEAR == 2018)|(vacate_df.YEAR == 2019)]
vacate1

,YEAR,VACATE_DATE,BOROUGH,NUMBER_OF_VACATE_ORDERS
0,2018,2018-01-01,BROOKLYN,30
1,2018,2018-01-01,BRONX,25
2,2018,2018-01-01,MANHATTAN,13
3,2018,2018-01-01,QUEENS,14
4,2018,2018-01-01,STATEN ISLAND,7
...,...,...,...,...
110,2019,2019-11-01,STATEN ISLAND,1
111,2019,2019-12-01,BROOKLYN,13
112,2019,2019-12-01,BRONX,10
113,2019,2019-12-01,MANHATTAN,3


In [53]:
vacate2 = vacate_df.loc[(vacate_df.YEAR == 2020) | (vacate_df.YEAR == 2021)]
vacate2

,YEAR,VACATE_DATE,BOROUGH,NUMBER_OF_VACATE_ORDERS
115,2020,2020-01-01,BROOKLYN,18
116,2020,2020-01-01,BRONX,16
117,2020,2020-01-01,MANHATTAN,8
118,2020,2020-01-01,QUEENS,8
119,2020,2020-02-01,BROOKLYN,22
...,...,...,...,...
190,2021,2021-05-01,BROOKLYN,8
191,2021,2021-05-01,BRONX,6
192,2021,2021-05-01,MANHATTAN,2
193,2021,2021-05-01,QUEENS,6


## Mapping alarm boxes

## Data Wrangling